# Garmin Analysis Notebook:

## Modules and Data Import:

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import plotly.graph_objects as go

# import OLS
import statsmodels.api as sm
from statsmodels.formula.api import ols

import calendar


plt.style.use('fast')

# dont show errors
import warnings
warnings.filterwarnings('ignore')


In [9]:
from Garmin_API_V1 import activitydata

AssertionError: 

In [ ]:
activitydata['date'] = pd.to_datetime(activitydata['date']).dt.date

activitydata = activitydata.assign(
    year=pd.DatetimeIndex(activitydata['date']).year,
    month=pd.DatetimeIndex(activitydata['date']).month
).query('pace > 0').query(
    'activityType == "running" or activityType == "trail_running" or activityType == "treadmill_running"'
)

# if elevationGain is null, set it to 0
activitydata['elevationGain'] = activitydata['elevationGain'].fillna(0)


In [ ]:

monthly = ( activitydata[(activitydata['year'] >= 2023)]
           .groupby(['year', 'month'])['distance']
           .sum()
           .reset_index()
)
# monthly.head(20)


## Analysis

### Monthly Progres:

#### Graphs:

In [ ]:
p = sns.barplot(x=monthly.month, y='distance', data=monthly, hue='year')
plt.ylabel('Total Distance (Miles)')
plt.xlabel('Month')
plt.legend(loc='upper right')  
plt.title('Total Distance Run Per Month')
plt.show()


In [ ]:
base_zone = (activitydata[(activitydata['year'] == 2024) & 
                         (activitydata['averageHR'] > 140) & 
                         (activitydata['averageHR'] < 150)]
            .assign(quarter=lambda x: pd.DatetimeIndex(x['date']).quarter)
            .query('~activityName.str.contains("Intervals")')
            .query('~activityName.str.contains("Hybrid")')
            .query('~activityName.str.contains("Hike")')
            .query('activityType != "trail_running"')
            .query('date != @dt.date(2024, 8, 31)')
            # query when activity is not treadmill and ascent is not greater than 100
            .query('elevationGain < 1000')

            )


In [ ]:
easy_zone = (activitydata[(activitydata['year'] == 2024) & 
                         (activitydata['averageHR'] > 120) & 
                         (activitydata['averageHR'] < 140)]
            .assign(quarter=lambda x: pd.DatetimeIndex(x['date']).quarter)
            .query('~activityName.str.contains("Intervals")')
            .query('~activityName.str.contains("Hybrid")')
            .query('~activityName.str.contains("Hike")')
            .query('activityType != "trail_running"')
            .query('date != @dt.date(2024, 8, 31)')
            .query('date != @dt.date(2024, 2, 28)')
            .query('elevationGain < 1000')
)




In [ ]:
# anova of pace for each month of the year
model = ols('pace ~ C(month)', data=easy_zone).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

# boxplot of pace for each month of the year
sns.boxplot(x='month', y='pace', data=easy_zone)

# lable means on the boxplot with offset
means = easy_zone.groupby('month')['pace'].mean().values
mean_labels = [str(np.round(mean, 2)) for mean in means]
pos = range(len(means))
for tick,label in zip(pos, plt.gca().get_xticklabels()):
    plt.text(pos[tick], means[tick] + 0.5, mean_labels[tick], 
             horizontalalignment='center', size='x-small', color='black', weight='semibold')

plt.ylabel('Pace (min/mile)')
plt.xlabel('Month')
plt.ylim(8, 13)
plt.title('Pace by Month for Heart Rate 110-140')
plt.show()


In [ ]:
# anova of pace for each month of the year
model = ols('pace ~ C(month)', data=base_zone).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

# boxplot of pace for each month of the year
sns.boxplot(x='month', y='pace', data=base_zone)

# lable means on the boxplot with offset
means = base_zone.groupby('month')['pace'].mean().values
mean_labels = [str(np.round(mean, 2)) for mean in means]
pos = range(len(means))
for tick,label in zip(pos, plt.gca().get_xticklabels()):
    plt.text(pos[tick], means[tick] + 0.5, mean_labels[tick], 
             horizontalalignment='center', size='x-small', color='black', weight='semibold')

plt.ylabel('Pace (min/mile)')
plt.xlabel('Month')
plt.ylim(7, 12)
plt.title('Pace by Month for Heart Rate 140-150')
plt.show()


In [ ]:
# Calculate the rolling means
rolling_mean_base_pace = base_zone['pace'].rolling(window=3).mean()
rolling_mean_easy_zone = easy_zone['pace'].rolling(window=3).mean()

# Create the figure
fig = go.Figure()

# Scatter plot for `base_zone`
fig.add_trace(go.Scatter(
    x=base_zone['date'],
    y=base_zone['pace'],
    mode='markers',
    marker=dict(opacity=0.5, color='orange'),
    name='Pace (Base HR 140-150)'
))

# Rolling mean for `base_zone`
fig.add_trace(go.Scatter(
    x=base_zone['date'],
    y=rolling_mean_base_pace,
    mode='lines',
    line=dict(color='orange'),
    name='Rolling Mean (Base HR 140-150)'
))

# Scatter plot for `easy_zone`
fig.add_trace(go.Scatter(
    x=easy_zone['date'],
    y=easy_zone['pace'],
    mode='markers',
    marker=dict(opacity=0.5, color='blue'),
    name='Pace (Easy HR 110-140)'
))

# Rolling mean for `easy_zone`
fig.add_trace(go.Scatter(
    x=easy_zone['date'],
    y=rolling_mean_easy_zone,
    mode='lines',
    line=dict(color='blue'),
    name='Rolling Mean (Easy HR 110-140)'
))

# Define vertical lines with annotations with text offset from the line
vlines = [
    {'date': pd.Timestamp('2024-02-01'), 'label': 'Covid', 'xshift': 5},
    {'date': pd.Timestamp('2024-05-08'), 'label': 'Sedona Canyons 125', 'xshift': 5},
    {'date': pd.Timestamp('2024-10-26'), 'label': 'Ozark Trail 100', 'xshift': 5}
]

# Add vertical lines and annotations
for vline in vlines:
    fig.add_vline(
        x=vline['date'],
        line=dict(color='red', dash='dash')
    )
    fig.add_annotation(
        x=vline['date'],
        y=12,  # Adjust y-position based on the plot range
        text=vline['label'],
        showarrow=False,
        yshift=10,
        textangle=90,
        font=dict(size=12)
    )

# Update layout
fig.update_layout(
    width=800,
    height=500,
    title='Combined Pace Over Time for HR Zones (Base HR 140-150 and Easy HR 110-140)',
    xaxis_title='Date',
    yaxis_title='Pace (min/mile)',
    xaxis=dict(tickangle=45),
    showlegend=True
)

# Display the figure
fig.show()

In [ ]:
easy_zone_monthly = (easy_zone
                    .groupby(['year', 'month'])['pace']
                    .mean()
                    .reset_index()
)

base_zone_monthly = (base_zone
                     .groupby(['year', 'month'])['pace']
                     .mean()
                     .reset_index()
)

easy_base_compare = ( pd.merge(easy_zone_monthly, base_zone_monthly, on=['year', 'month'], suffixes=('_easy', '_base'))
                        .assign(pace_diff=lambda x: x['pace_easy'] - x['pace_base'])
    )


In [ ]:
easy_base_compare.head(12)

In [ ]:


# Assuming 'easy_base_compare' DataFrame is already defined
# and contains 'month', 'pace_easy', and 'pace_base' columns.

# Calculate angles for each month
easy_base_compare['angles'] = (easy_base_compare['month'] / 12) * 2 * np.pi

# Define bar width and offsets
width = (2 * np.pi) / 12 * 0.45  # 45% of the space for each bar
offset = width / 2                # Offset to separate bars

# Initialize the polar subplot
fig = plt.figure(figsize=(10, 8))
ax = plt.subplot(111, polar=True)

# Set radial limits and invert the radial axis
min_pace = min(easy_base_compare['pace_easy'].min(), easy_base_compare['pace_base'].min())
max_pace = max(easy_base_compare['pace_easy'].max(), easy_base_compare['pace_base'].max())

# Extend limits for padding and invert the axis
ax.set_ylim(min_pace - 1, max_pace + 1)
ax.set_ylim(ax.get_ylim()[::-1])  # Invert the radial axis

# Bars for pace_easy
bars_easy = ax.bar(
    easy_base_compare['angles'] - offset,
    easy_base_compare['pace_easy'],
    width=width,
    bottom=0.0,
    color='skyblue',
    edgecolor='black',
    linewidth=1,
    label='Easy Pace'
)

# Bars for pace_base
bars_base = ax.bar(
    easy_base_compare['angles'] + offset,
    easy_base_compare['pace_base'],
    width=width,
    bottom=0.0,
    color='coral',
    edgecolor='black',
    linewidth=1,
    label='Base Pace'
)

# Set the labels for each bar (month names)
all_months = np.arange(1, 13)
all_angles = (all_months / 12) * 2 * np.pi
all_month_names = [calendar.month_abbr[m] for m in all_months]

ax.set_xticks(all_angles)
ax.set_xticklabels(all_month_names)

# Adjust the orientation
ax.set_theta_offset(np.pi / 2)  # Start from the top
ax.set_theta_direction(-1)      # Clockwise

# Set yticks and labels (inverted axis)
yticks = np.arange(np.floor(min_pace - 1), np.ceil(max_pace + 1), 0.5)
ax.set_yticks(yticks)
ax.set_yticklabels([f"{tick:.1f}" for tick in yticks])

# Add data labels for 'pace_easy'
for angle, value in zip(easy_base_compare['angles'] - offset, easy_base_compare['pace_easy']):
    ax.text(
        angle,
        value - 0.2,  # Adjust position due to inversion
        f"{value:.2f}",
        ha='center',
        va='top',
        fontsize=9
    )

# Add data labels for 'pace_base'
for angle, value in zip(easy_base_compare['angles'] + offset, easy_base_compare['pace_base']):
    ax.text(
        angle,
        value - 0.2,
        f"{value:.2f}",
        ha='center',
        va='top',
        fontsize=9
    )

# Add title and legend
plt.title('Easy Pace vs Base Pace by Month (Improvement Highlighted)', y=1.08)
ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1))

# Add gridlines
ax.grid(True)

# Show the plot
plt.tight_layout()
plt.show()
